In [1]:
%cd descriptors/

/Users/mike/pya/Konferenzen/EuroPython/EuroPython2025/tutorial/descriptors


In [3]:
# %load datadescriptor.py
"""A typical data descriptor."""


class DataDescriptor:
    """A simple descriptor."""

    def __init__(self):
        self.value = 0

    def __get__(self, instance, cls):
        print('data descriptor __get__')
        return self.value

    def __set__(self, instance, value):
        print('data descriptor __set__')
        try:
            self.value = value.upper()
        except AttributeError:
            self.value = value

    def __delete__(self, instance):
        print("Don't like to be deleted.")


In [4]:
class WithDescriptor:
    attr = DataDescriptor()

In [5]:
w1 = WithDescriptor()

In [6]:
w1.attr

data descriptor __get__


0

In [7]:
w1.attr = 'hello'

data descriptor __set__


In [8]:
w1.attr

data descriptor __get__


'HELLO'

In [11]:
type(w1).__dict__['attr'].__get__(w1, type(w1))

data descriptor __get__


'HELLO'

In [12]:
w2 = WithDescriptor()

In [13]:
w2.attr

data descriptor __get__


'HELLO'

In [14]:
w2.__dict__

{}

In [15]:
w2.attr = 200

data descriptor __set__


In [16]:
w2.__dict__

{}

In [17]:
w1.attr

data descriptor __get__


200

In [18]:
del w1.attr

Don't like to be deleted.


In [20]:
# %load nondatadescriptor.py
"""A non data descriptor."""


class NonDataDescriptor:
    """Non data descriptor don't implement `__set__`."""

    def __init__(self):
        self.value = 0

    def __get__(self, instance, cls):
        print('non data descriptor __get__')
        return self.value + 10


In [21]:
class AddTen:
    attr = NonDataDescriptor()

In [22]:
a10 = AddTen()

In [23]:
a10.attr

non data descriptor __get__


10

In [24]:
a10.__dict__

{}

In [25]:
a10.attr = 5

In [26]:
a10.attr

5

In [27]:
a10.__dict__

{'attr': 5}

In [28]:
%ls

cache.py                      percent.py
checked.py                    protected_deco.py
class_storage.py              protected.py
datadescriptor.py*            read_only.py
example_sqla.py               special.py
instance_storage.py           unprotected.py
nondatadescriptor.py*         weakkeydict_storage_break.py
percent_string_name.py        weakkeydict_storage.py


In [35]:
class A:

    def __init__(self, value):
        self.value = value

    def __getattr__(self, name):
        return 42

In [36]:
a = A(10)

In [37]:
a.value

10

In [38]:
a.xgxgx

42

In [39]:
class B:

    def __init__(self, value):
        self.value = value

    def __getattribute__(self, name):
        return 42

## Doesn't work

```python
B(2)
```

## Functions are Non-Data Descriptors

In [41]:
def f():
    ...

In [42]:
f.__get__

<method-wrapper '__get__' of function object at 0x10c16e8e0>

In [47]:
class C:

    def meth(self):
        return 42

In [48]:
inst = C()

In [49]:
inst.meth

<bound method C.meth of <__main__.C object at 0x1061f1d30>>

In [50]:
C.meth

<function __main__.C.meth(self)>

In [51]:
inst.meth()

42

In [53]:
C.meth(inst)

42

In [54]:
type(inst).__dict__['meth'].__get__(inst, type(inst))

<bound method C.meth of <__main__.C object at 0x1061f1d30>>

In [56]:
C.meth(34)

42

In [57]:
class D:

    def __init__(self, value):
        self.value = value

    def meth(self):
        return self.value

In [59]:
d = D(10)

In [60]:
d.meth()

10

In [61]:
D.meth(d)

10

In [62]:
D.meth(34)

AttributeError: 'int' object has no attribute 'value'

In [66]:
# %load class_storage.py

"""
A descriptor works only in a class.

Storing attribute data directly in a descriptor
means sharing between instances.
"""


class DescriptorClassStorage:
    """Descriptor storing data in class."""

    def __init__(self, default=None):
        self.value = default

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        self.value = value


def test():
    """Demonstrate class storage."""

    class StoreClass:
        """All instances will share `attr`."""

        attr = DescriptorClassStorage(10)

    store1 = StoreClass()
    store2 = StoreClass()
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('Setting store1 only.')
    store1.attr = 100
    print('store1', store1.attr)
    print('store2', store2.attr)



In [68]:
class StoreClass:
    """All instances will share `attr`."""

    attr = DescriptorClassStorage(10)

In [69]:
store1 = StoreClass()
store2 = StoreClass()

In [70]:
store1.attr

10

In [71]:
store2.attr

10

In [72]:
store1.attr = 100

In [73]:
store2.attr

100

In [74]:
store1.__dict__

{}

In [76]:
# %load weakkeydict_storage.py

"""
A descriptor works only in a class.

We can store a different value for each instance in a dictionary
in the descriptor.
"""

from weakref import WeakKeyDictionary


class DescriptorWeakKeyDictStorage:
    """Descriptor that stores attribute data in instances."""

    def __init__(self, default=None):
        self._hidden = WeakKeyDictionary()
        self.default = default

    def __get__(self, instance, owner):
        return self._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        self._hidden[instance] = value


def test():
    """Demonstrate instance storage."""

    class StoreInstance:
        """All instances have own `attr`."""

        attr1 = DescriptorWeakKeyDictStorage(10)
        attr2 = DescriptorWeakKeyDictStorage(-5)

    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr1)
    print('store2', store2.attr1)
    print('Setting store1 only.')
    store1.attr1 = 100
    store1.attr2 = -123
    store2.attr1 = 98765
    print('store1 attr1', store1.attr1)
    print('store1 attr2', store1.attr2)
    print('store2', store2.attr1)
    print('_hidden:', list(StoreInstance.__dict__['attr1']._hidden.items()))
    print('_hidden:', list(StoreInstance.__dict__['attr2']._hidden.items()))
    del store1
    print('Deleted store1')
    print('_hidden:', list(StoreInstance.__dict__['attr1']._hidden.items()))




In [77]:
class StoreInstance:
    """All instances have own `attr`."""

    attr1 = DescriptorWeakKeyDictStorage(10)
    attr2 = DescriptorWeakKeyDictStorage(-5)

In [78]:
store1 = StoreInstance()
store2 = StoreInstance()

In [79]:
store1.attr1

10

In [80]:
store2.attr1

10

In [81]:
store1.attr1 = 100

In [82]:
store1.attr1

100

In [83]:
store2.attr1

10

In [88]:
dict(StoreInstance.__dict__['attr1']._hidden)

{<__main__.StoreInstance at 0x1061f1550>: 100}

In [90]:
hex(hash(store1))

'0x1061f155'

In [91]:
# %load weakkeydict_storage_break.py
"""
Break DescriptorWeakKeyDictStorage

Making your class non-hashable breaks the weakkey storage.
"""

from weakkeydict_storage import DescriptorWeakKeyDictStorage


class StoreInstance:
    """All instances have own `attr`."""

    attr1 = DescriptorWeakKeyDictStorage(10)
    attr2 = DescriptorWeakKeyDictStorage(-5)

    def __hash__(self):
        raise NotImplementedError

    # def __eq__(self, other):
    #    raise False


def test():
    """Show that is doesn't work anymore."""
    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr1)
    print('store2', store2.attr1)
    print('Setting store1 only.')
    store1.attr1 = 100
    store1.attr2 = -123
    store2.attr1 = 98765
    print('store1 attr1', store1.attr1)
    print('store1 attr2', store1.attr2)
    print('store2', store2.attr1)
    print('_hidden:', list(StoreInstance.__dict__['attr1']._hidden.items()))
    print('_hidden:', list(StoreInstance.__dict__['attr2']._hidden.items()))
    del store1
    print('Deleted store1')
    print('_hidden:', list(StoreInstance.__dict__['attr1']._hidden.items()))


if __name__ == '__main__':
    test()


In [93]:
# %load instance_storage.py

"""
A descriptor works only in a class.

Storing attribute data in `instance.__dict__` can be a solution.
Creating a unique attribute name with the help of `uuid` and
using name mangling with the `__` prefix make things more robust.
"""

import uuid


class DescriptorInstanceStorage:
    """Descriptor that stores attribute data in instances."""

    def __init__(self, default=None):
        # unique name with uuid
        self._hidden_name = '__' + uuid.uuid4().hex
        self.default = default

    def __get__(self, instance, owner):
        return getattr(instance, self._hidden_name, self.default)

    def __set__(self, instance, value):
        setattr(instance, self._hidden_name, value)


def test():
    """Demonstrate how it works."""

    class StoreInstance:
        """All instances have own `attr`."""

        attr = DescriptorInstanceStorage(10)

    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('Setting store1 only.')
    store1.attr = 100
    print('store1', store1.attr)
    print('store2', store2.attr)
    print(store1.__dict__)




In [96]:
uuid.uuid4().hex

'dcb6c697ad954de7a0c8711caf5420df'

In [97]:
class E:

    def _internal(self):
        return 42

    def __secret(self):
        return 43

In [98]:
e = E()

In [100]:
e._internal()

42

In [102]:
e._E__secret()

43

In [103]:
e._internal

<bound method E._internal of <__main__.E object at 0x106066e40>>

In [104]:
getattr(e, '_internal')

<bound method E._internal of <__main__.E object at 0x106066e40>>

In [106]:
getattr(e, 'x_internal', 999)

999

In [107]:
class StoreInstance:
    """All instances have own `attr`."""

    attr = DescriptorInstanceStorage(10)

In [108]:
store1 = StoreInstance()
store2 = StoreInstance()

In [109]:
store1.attr

10

In [110]:
store2.attr

10

In [111]:
store1.attr = 100

In [112]:
store2.attr

10

In [113]:
dir(store1)

['__cb9ddbb329d946aea188499c0b54c316',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attr']

In [114]:
%ls

cache.py                      percent.py
checked.py                    protected_deco.py
class_storage.py              protected.py
datadescriptor.py*            read_only.py
example_sqla.py               special.py
instance_storage.py           unprotected.py
nondatadescriptor.py*         weakkeydict_storage_break.py
percent_string_name.py        weakkeydict_storage.py


In [117]:
int('2')

2

In [118]:
type(int)

type

In [119]:
# %load checked.py


"""Example for descriptor that checks conditions on attributes."""

import uuid


class Checked:
    """
    Check valid attributes.

    Descriptor that checks with a user-supplied check function
    if an attribute is valid.
    """

    def __init__(self, checker=None, default=None):
        if checker:
            # checker must be a callable
            checker(default)
        self.checker = checker
        self.default = default
        self._hidden_name = '__' + uuid.uuid4().hex

    def __get__(self, instance, owner):
        return getattr(instance, self._hidden_name, self.default)

    def __set__(self, instance, value):
        if self.checker:
            self.checker(value)
        setattr(instance, self._hidden_name, value)


def is_int(value):
    """Check if value is an integer."""
    if not isinstance(value, int):
        raise ValueError(f'Int required {type(value)} found')


class Restricted:
    """Use checked attributes."""

    attr1 = Checked(checker=is_int, default=10)
    attr2 = Checked(default=12.5)
    # Setting the default to float, `is_int` raises a `ValueError`.
    try:
        attr3 = Checked(checker=is_int, default=12.5)
    except ValueError:
        print('Cannot set default to float, must be int.')
        attr3 = Checked(checker=is_int, default=12)


def test():
    """Demonstrate how it works."""
    restricted = Restricted()
    print('attr1', restricted.attr1)
    restricted.attr1 = 100
    print('attr1', restricted.attr1)
    try:
        restricted.attr1 = 200.12
    except ValueError:
        print('Cannot set attr1 to float, must be int.')
        restricted.attr1 = 200
    print(restricted.attr1, restricted.attr2, restricted.attr3)




Cannot set default to float, must be int.


In [120]:
restricted = Restricted()

In [121]:
restricted.attr1

10

In [122]:
restricted.attr1 = 100

In [123]:
restricted.attr1

100

In [124]:
restricted.attr1 = 1.1

ValueError: Int required <class 'float'> found

In [125]:
%ls

cache.py                      percent.py
checked.py                    protected_deco.py
class_storage.py              protected.py
datadescriptor.py*            read_only.py
example_sqla.py               special.py
instance_storage.py           unprotected.py
nondatadescriptor.py*         weakkeydict_storage_break.py
percent_string_name.py        weakkeydict_storage.py


In [127]:
# %load percent.py
"""Example for the Same-Name-Trick."""


class Percent:
    """Descriptor for percent values between 0 and 100."""

    def __set_name__(self, owner, attr_name):
        self.attr_name = attr_name

    def __set__(self, instance, value):
        if not 0 <= value <= 100:
            raise ValueError(f'Value must be between 0 and 100. Got: {value}.')
        instance.__dict__[self.attr_name] = value


class WithPercent:
    """Class with attribute between 0 and 100."""

    percent = Percent()

    def __init__(self, percent):
        self.percent = percent


In [128]:
p = WithPercent(34)

In [129]:
p.percent

34

In [130]:
p.__dict__

{'percent': 34}

In [131]:
p.percent = 28

In [132]:
p.percent

28

In [133]:
p.__dict__

{'percent': 28}

In [134]:
p.percent = -6

ValueError: Value must be between 0 and 100. Got: -6.

In [135]:
%ls

cache.py                      percent.py
checked.py                    protected_deco.py
class_storage.py              protected.py
datadescriptor.py*            read_only.py
example_sqla.py               special.py
instance_storage.py           unprotected.py
nondatadescriptor.py*         weakkeydict_storage_break.py
percent_string_name.py        weakkeydict_storage.py


In [137]:
# %load read_only.py
"""Read only attributes."""


class WrongReadOnly:
    """Wrong: missing `__set__` that raises exception."""

    def __set_name__(self, owner, attr_name):
        self.attr_name = attr_name

    def __get__(self, instance, owner):
        return 42


class Wrong:
    """Use `WrongReadOnly`."""

    attr_wrong_ro = WrongReadOnly()


class RightReadOnly:
    """Right: `_set__` raises exception."""

    def __set_name__(self, owner, attr_name):
        self.attr_name = attr_name

    def __get__(self, instance, owner):
        return 42

    def __set__(self, instance, value):
        raise AttributeError(f'attribute "{self.attr_name}" is read-only')


class Right:
    """Use `RightReadOnly`."""

    attr_ro = RightReadOnly()


In [138]:
w = Wrong()

In [139]:
w.attr_wrong_ro

42

In [140]:
w.attr_wrong_ro = 34

In [141]:
w.attr_wrong_ro

34

In [142]:
r = Right()

In [143]:
r.attr_ro

42

In [144]:
r.attr_ro = 34

AttributeError: attribute "attr_ro" is read-only

In [145]:
w.__dict__

{'attr_wrong_ro': 34}

In [146]:
r.__dict__

{}

In [148]:
# %load cache.py
"""Caching attributes."""

class Cached:
    """Cached attribute."""

    def __set_name__(self, owner, attr_name):
        self.attr_name = attr_name

    def __get__(self, instance, owner):
        if instance is None:
            raise ValueError
        print('doing expensive calculation at first access')
        # do your work here
        value = 42
        instance.__dict__[self.attr_name] = value
        return value


class WithCache:
    """`attr` is calculated only one."""

    attr = Cached()


In [149]:
wc = WithCache()

In [150]:
wc.__dict__

{}

In [151]:
wc.attr

doing expensive calculation at first access


42

In [153]:
wc.__dict__

{'attr': 42}

In [154]:
%ls

cache.py                      percent.py
checked.py                    protected_deco.py
class_storage.py              protected.py
datadescriptor.py*            read_only.py
example_sqla.py               special.py
instance_storage.py           unprotected.py
nondatadescriptor.py*         weakkeydict_storage_break.py
percent_string_name.py        weakkeydict_storage.py


In [156]:
# %load special.py
"""Specal methods are special."""


class Special:
    """Class with special method."""

    def __contains__(self, item):
        print('got', item)
        return True


def test():
    """Demonstarte how it works."""
    special = Special()
    print('before:', 'abc' in special)
    special.__contains__ = 43
    print('after:', 'abc' in special)




In [157]:
special = Special()

In [158]:
special.__dict__

{}

In [160]:
2 in special

got 2


True

In [159]:
special.__contains__(2)

got 2


True

In [161]:
special.__contains__ = 100

In [162]:
2 in special

got 2


True

In [163]:
special.__contains__(2)

TypeError: 'int' object is not callable

In [164]:
special.__dict__

{'__contains__': 100}

In [166]:
# %load unprotected.py
"""Normal methods can be overridden."""


class Unprotected:
    """Class with a method."""

    def meth(self):
        """Do something."""
        print('called meth')
        return 42


def test():
    """Demonstrate."""
    unprotected = Unprotected()
    print('before:', unprotected.meth())
    unprotected.meth = 43
    print('after:', unprotected.meth)




In [167]:
unprotected = Unprotected()

In [168]:
unprotected.meth()

called meth


42

In [169]:
unprotected.meth = 43

In [180]:
unprotected.meth

43

In [170]:
unprotected.meth()

TypeError: 'int' object is not callable

In [172]:
# %load protected.py
"""Protect methods from being overridden."""

import types


class Protector:
    """Descriptor that protects methods from overriding."""

    def __init__(self, meth, use_cache):
        self.meth = meth
        self.use_cache = use_cache

    def __get__(self, instance, cls):
        is_static_method = isinstance(self.meth, staticmethod)
        # no method binding if
        # * access via class --> instance is None or
        # * method is a classmethod --> instance is None or
        # * method is a static method
        if instance is None or is_static_method:
            return self.meth

        def bind_method():
            """Bind method to instance and print message."""
            print(f'binding {self.meth.__name__}')
            return types.MethodType(self.meth, instance)

        if self.use_cache:
            bound_meth = instance.__dict__.get(self.meth.__name__)
            if not bound_meth:
                bound_meth = bind_method()
                instance.__dict__[self.meth.__name__] = bound_meth
        else:
            bound_meth = bind_method()
        return bound_meth

    def __set__(self, instance, value):
        raise AttributeError('cannot set method')


def protect_methods(use_cache):
    """Class decorator to apply `Protector`."""

    def _protect_methods(cls):
        """Replace methods with descriptors."""
        for name, obj in cls.__dict__.items():
            if callable(obj):
                setattr(cls, name, Protector(obj, use_cache=use_cache))
        return cls

    return _protect_methods


In [174]:
# %load protected_deco.py
"""Use class decorator `protect_methods`."""

from protected import protect_methods


@protect_methods(use_cache=True)
class ProtectedCached:
    """Class with protected methods."""

    def meth1(self):
        """Test method 1."""
        print('called')
        return 42

    def meth2(self):
        """Test method 2."""
        print('called')
        return 142

    @staticmethod
    def func():
        """Provide a static method."""
        return 42

    @classmethod
    def cls_meth(cls):
        """Provide a class method."""
        return 42


@protect_methods(use_cache=False)
class ProtectedUncached:
    """Class with protected methods."""

    def meth1(self):
        """Test method 1."""
        print('called')
        return 42

    def meth2(self):
        """Test method 2."""
        print('called')
        return 142

    @staticmethod
    def func():
        """Provide a static method."""
        return 42

    @classmethod
    def cls_meth(cls):
        """Provide a class method."""
        return 42


In [175]:
pc = ProtectedCached()

In [176]:
pc.__dict__

{}

In [177]:
pc.meth1()

called


42

In [178]:
pc.__dict__

{'meth1': <bound method ProtectedCached.meth1 of <__main__.ProtectedCached object at 0x10b94dfd0>>}

In [179]:
pc.meth1 = 34

AttributeError: cannot set method

In [183]:
number = -5

if number < 0:
    raise ValueError(f'number {number} is too small')

ValueError: number -5 is too small

In [184]:
%cd ../solutions_source/descriptors/

/Users/mike/pya/Konferenzen/EuroPython/EuroPython2025/tutorial/solutions_source/descriptors


In [185]:
%ls

ex1.py* ex2.py* ex3.py* ex4.py*


In [187]:
# %load ex4.py
"""Prices with VAT."""

#  pylint: disable-msg=too-few-public-methods


class Total:
    """Allow only positive values."""

    def __init__(self, rate=1.19):
        self.rate = rate

    def __get__(self, instance, cls):
        return round(instance.net * self.rate, 2)

    def __set__(self, instance, value):
        raise NotImplementedError('Cannot change value.')


class Price:
    """A price with VAT."""

    def __init__(self, net):
        self.net = net


class PriceGermany(Price):
    """A price with VAT for Germany."""

    total = Total(1.19)


class PriceDenmark(Price):
    """A price with VAT for Denmark."""

    total = Total(1.25)


class PriceLuxembourg(Price):
    """A price with VAT Luxembourg."""

    total = Total(1.17)


if __name__ == '__main__':

    def test():
        """Demonstrate how it works."""
        price_germany = PriceGermany(110)
        print('Germany', price_germany.total)

        price_denmark = PriceDenmark(110)
        print('Denmark', price_denmark.total)

        price_luxembourg = PriceLuxembourg(110)
        print('Luxembourg', price_luxembourg.total)

        # Try to set it.
        price_denmark.total = 120

    test()


Germany 130.9
Denmark 137.5
Luxembourg 128.7


NotImplementedError: Cannot change value.

In [189]:

class Total:
    """Allow only positive values."""

    def __init__(self, rate=1.19, net_name='net'):
        self.rate = rate
        self.net_name = net_name

    def __get__(self, instance, cls):
        return round(getattr(instance, self.net_name) * self.rate, 2)

    def __set__(self, instance, value):
        raise NotImplementedError('Cannot change value.')


class Price:
    """A price with VAT."""

    def __init__(self, net):
        self.net2 = net


class PriceGermany(Price):
    """A price with VAT for Germany."""

    total = Total(1.19, net_name='net2')

In [191]:
pg = PriceGermany(34)

In [192]:
pg.total

40.46